In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import io
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import math

In [0]:
df1 = pd.read_csv('train.csv')
df2 = pd.read_csv('test.csv')

In [0]:
df1.head()

,estimated blast radius(km),temperature(C),distance from nearest carrier(nm),military-to-civilian ratio,population density,relative humidity,squadron strength,city,Id
0,1.51662,12.85,3.51,1.44,73.01,0.68,8.23,2,0
1,1.52614,13.70,0.00,1.36,71.24,0.19,13.44,2,2
2,1.51747,12.84,3.50,1.14,73.27,0.56,8.55,1,5
3,1.51590,12.82,3.52,1.90,72.86,0.69,7.97,2,7
4,1.51768,12.65,3.56,1.30,73.08,0.61,8.69,1,8


In [0]:
replace_map = {'city': {1:0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6}}
df1.replace(replace_map, inplace=True)
df2.replace(replace_map, inplace=True)

In [0]:
df1.head()

,estimated blast radius(km),temperature(C),distance from nearest carrier(nm),military-to-civilian ratio,population density,relative humidity,squadron strength,city,Id
0,1.51662,12.85,3.51,1.44,73.01,0.68,8.23,1,0
1,1.52614,13.70,0.00,1.36,71.24,0.19,13.44,1,2
2,1.51747,12.84,3.50,1.14,73.27,0.56,8.55,0,5
3,1.51590,12.82,3.52,1.90,72.86,0.69,7.97,1,7
4,1.51768,12.65,3.56,1.30,73.08,0.61,8.69,0,8


In [0]:
Y = df1['city'].copy()
X = df1[['estimated blast radius(km)','temperature(C)','distance from nearest carrier(nm)','military-to-civilian ratio','population density','relative humidity','squadron strength']].copy()
X_test = df2[['estimated blast radius(km)','temperature(C)','distance from nearest carrier(nm)','military-to-civilian ratio','population density','relative humidity','squadron strength']].copy()

In [0]:
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_test = scaler.transform(X_test)

In [0]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.30)

In [0]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import confusion_matrix, accuracy_score
dtree_model = DecisionTreeClassifier(max_depth = 2).fit(X_train, Y_train) 
dtree_predictions = dtree_model.predict(X_val) 
  
# creating a confusion matrix 
cm = confusion_matrix(Y_val, dtree_predictions) 
cm

accuracy = accuracy_score(Y_val,dtree_predictions)
accuracy

0.5853658536585366

In [0]:
from sklearn.svm import SVC 
svm_model_linear = SVC(kernel = 'linear', C = 1).fit(X_train, Y_train) 
svm_predictions = svm_model_linear.predict(X_val) 
  
# model accuracy for X_test   
accuracy = svm_model_linear.score(X_val, Y_val) 
accuracy

0.6097560975609756

In [0]:
from sklearn.neighbors import KNeighborsClassifier 
knn = KNeighborsClassifier(n_neighbors = 7).fit(X_train, Y_train) 
  
# accuracy on X_test 
accuracy = knn.score(X_val, Y_val) 
accuracy  

0.5853658536585366

In [0]:
from sklearn.naive_bayes import GaussianNB 
gnb = GaussianNB().fit(X_train, Y_train) 
gnb_predictions = gnb.predict(X_val) 
  
# accuracy on X_test 
accuracy = gnb.score(X_val, Y_val) 
accuracy 

0.4634146341463415

In [0]:
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators=100)
clf.fit(X_train,Y_train)

accuracy = clf.score(X_val, Y_val)
accuracy

0.6097560975609756

In [0]:
P = clf.predict(X_test)
P

array([1, 0, 1, 0, 0, 6, 4, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 6, 1, 1, 1,
       1, 6, 1, 0, 6, 1, 4, 1, 2, 0, 6, 6, 6, 1, 1, 1, 0, 0, 0, 0, 2, 1,
       0, 0, 0, 0, 0, 1, 1, 1, 2, 0, 0, 1, 1, 4, 6, 0, 0, 1, 4, 0, 6, 6,
       1, 6, 6, 0, 1, 1, 5, 6, 2, 0, 0, 1, 1])

In [0]:
P = P.flatten()
id = df2['Id'].to_list()
Y_pred = pd.DataFrame(list(zip(id, P)), 
               columns =['Id', 'city']) 
replace_map = {'city': {0:1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7}}
Y_pred.replace(replace_map, inplace=True)
Y_pred.set_index(['Id'], inplace=True)
Y_pred.to_csv('city.csv')

In [0]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Input(shape=(7,)),
  #tf.keras.layers.Dropout(0.8),
  tf.keras.layers.Dense(14, activation='relu'),
  tf.keras.layers.Dense(7,activation='softmax')
])

model.compile(optimizer='Adam',
      loss='categorical_crossentropy', metrics=['accuracy'])
  #return model
  
r = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=200)

Epoch 1/200
3/3 [==============================] - 0s 36ms/step - loss: 0.0000e+00 - accuracy: 0.3511 - val_loss: 0.0000e+00 - val_accuracy: 0.3171
Epoch 2/200
3/3 [==============================] - 0s 12ms/step - loss: 0.0000e+00 - accuracy: 0.3511 - val_loss: 0.0000e+00 - val_accuracy: 0.3171
Epoch 3/200
3/3 [==============================] - 0s 12ms/step - loss: 0.0000e+00 - accuracy: 0.3511 - val_loss: 0.0000e+00 - val_accuracy: 0.3171
Epoch 4/200
3/3 [==============================] - 0s 12ms/step - loss: 0.0000e+00 - accuracy: 0.3511 - val_loss: 0.0000e+00 - val_accuracy: 0.3171
Epoch 5/200
3/3 [==============================] - 0s 14ms/step - loss: 0.0000e+00 - accuracy: 0.3511 - val_loss: 0.0000e+00 - val_accuracy: 0.3171
Epoch 6/200
3/3 [==============================] - 0s 12ms/step - loss: 0.0000e+00 - accuracy: 0.3511 - val_loss: 0.0000e+00 - val_accuracy: 0.3171
Epoch 7/200
3/3 [==============================] - 0s 11ms/step - loss: 0.0000e+00 - accuracy: 0.3511 - val_loss

In [0]:
P = model.predict_proba(X_test)
P = np.round(P).flatten()
print(P)

In [0]:
P = P.tolist()
Q = []
for i in P:
  Q.append(math.trunc(i))
Q

In [0]:
id = df2['Id'].to_list()
Y_pred = pd.DataFrame(list(zip(id, Q)), 
               columns =['Id', 'city']) 
Y_pred

,Id,city
0,1,0
1,3,0
2,4,0
3,6,0
4,9,0
...,...,...
74,203,0
75,204,0
76,206,0
77,209,0


In [0]:
replace_map = {'city': {0:1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7}}
Y_pred.replace(replace_map, inplace=True)

In [0]:
Y_pred

,Id,city
0,1,1
1,3,1
2,4,1
3,6,1
4,9,1
...,...,...
74,203,1
75,204,1
76,206,1
77,209,1


In [0]:
Y_pred.set_index(['Id'], inplace=True)
Y_pred

,city
Id,
1,1
3,1
4,1
6,1
9,1
...,...
203,1
204,1
206,1


In [0]:
Y_pred.to_csv('city.csv')